![x](https://zdnet4.cbsistatic.com/hub/i/r/2017/12/17/e9b8f576-8c65-4308-93fa-55ee47cdd7ef/resize/370xauto/30f614c5879a8589a22e57b3108195f3/databricks-logo.png)

&copy; 2019 Databricks, Inc. All rights reserved.<br/>

##MLflow Tutorial: Serving Models with Microsoft Azure ML

In this tutorial, we will use our MLflow model to deploy it to Azure ML for real-time serving.

This guide consists of the following sections:

#### Setup
* Launch an Azure Databricks cluster
* Install MLflow
* Install the Azure ML SDK
* Create or load an Azure ML Workspace
* (Optional) Connect to an MLflow tracking server

#### Building an Azure Container Image for model deployment
* Use MLflow to build a Container Image for the trained model

#### Deploying the model to "dev" using Azure Container Instances (ACI)
* Create an ACI webservice deployment using the model's Container Image

#### Querying the deployed model in "dev"
* Load a sample input vector
* Evaluate the sample input vector by sending an HTTP request

#### Deploying the model to production using Azure Kubernetes Service (AKS)
* Option 1: Create a new AKS cluster
* Option 2: Connect to an existing AKS cluster
* Deploy to the model's image to the specified AKS cluster

#### Querying the deployed model in production
* Load a sample input vector
* Evaluate the sample input vector by sending an HTTP request

#### Updating the production deployment
* Train a new model
* Build an Azure Container Image for the new model
* Deploy the new model's image to the AKS cluster
* Query the updated model

#### Cleaning up the deployments
* Terminate the "dev" ACI webservice
* Terminate the production AKS webservice
* Remove the AKS cluster from the Azure ML Workspace

### Install the Azure ML SDK

Once a cluster has been launched with the configuration described in **Launch an Azure Databricks cluster**, install the Azure Machine Learning SDK using the following steps:

1. Create the library with the Source ``Upload Python Egg or PyPI`` and the Pip library name:
  - `azureml-sdk[databricks]`     
     
2. Attach the library to the cluster.

### Create or load an Azure ML Workspace

Before models can be deployed to Azure ML, an Azure ML Workspace must be created or obtained. The `azureml.core.Workspace.create()` function will load a workspace of a specified name or create one if it does not already exist. For more information about creating an Azure ML Workspace, see the [Azure ML Workspace management documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-workspace).

In [7]:
import azureml
from azureml.core import Workspace

# workspace_name = "<WORKSPACE_NAME>"
# workspace_location="<WORKSPACE_LOCATION>"
# resource_group = "<RESOURCE_GROUP>"
# subscription_id = "<SUBSCRIPTION_ID>"

workspace_name = "workshop_adb_mlservice-aml"
workspace_location = "eastus2"
resource_group = "silv-workshop-infra-rg"
subscription_id = str(dbutils.secrets.get("workshop_secrets", "subscription-id"))

workspace = Workspace.create(name = workspace_name,
                             subscription_id = subscription_id,
                             resource_group = resource_group,
                             location = workspace_location,
                             exist_ok=True)

## Training a model

In [9]:
#We've already created a model, so let's just take the MLFlow URI:

model_uri = "dbfs:/databricks/mlflow/1622123275403635/d68e1ad0c43e4610a44496a8dd272850/artifacts/lrModelSK/"

## Building an Azure Container Image for model deployment

### Use MLflow to build a Container Image for the trained model

We will use the `mlflow.azuereml.build_image` function to build an Azure Container Image for the trained MLflow model. This function also registers the MLflow model with a specified Azure ML workspace. The resulting image can be deployed to Azure Container Instances (ACI) or Azure Kubernetes Service (AKS) for real-time serving.

In [12]:
import mlflow.azureml

model_image, azure_model = mlflow.azureml.build_image(model_uri=model_uri, 
                                                      workspace=workspace, 
                                                      model_name="cph-model",
                                                      image_name="cph-model-container-image",
                                                      description="skl for scoring machines",
                                                      synchronous=False)

Registering model cph-model
Creating image

In [13]:
model_image.wait_for_creation(show_output=True)

Running.....................................................................
Succeeded
Image creation operation finished for image cph-model-container-image:2, operation "Succeeded"

## Deploying the model to "dev" using [Azure Container Instances (ACI)](https://docs.microsoft.com/en-us/azure/container-instances/)

The [ACI platform](https://docs.microsoft.com/en-us/azure/container-instances/) is the recommended environment for staging and developmental model deployments.

### Create an ACI webservice deployment using the model's Container Image

Using the Azure ML SDK, we will deploy the Container Image that we built for the trained MLflow model to ACI.

In [16]:
from azureml.core.webservice import AciWebservice, Webservice

dev_webservice_name = "cph-model-dev"
dev_webservice_deployment_config = AciWebservice.deploy_configuration()
dev_webservice = Webservice.deploy_from_image(name=dev_webservice_name, image=model_image, deployment_config=dev_webservice_deployment_config, workspace=workspace)

Creating service

In [17]:
dev_webservice.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"

## Querying the deployed model in "dev"

### Load a sample input vector from the dataset

In [20]:
import numpy as np
import pandas as pd
from sklearn import datasets

sparkDF = spark.read.format("delta").load("/databricks_workshop/validate_clean").limit(20)

train_x = sparkDF.toPandas()
sample = train_x.iloc[:1, 1:]
sample_json = sample.to_json(orient="split")
query_input = list(sample.as_matrix().flatten())

In [21]:
sample_json

Out[ 23 ]: '{"columns":["IsBeta","RtpStateBitfield","IsSxsPassiveMode","AVProductStatesIdentifier","AVProductsInstalled","AVProductsEnabled","HasTpm","CountryIdentifier","CityIdentifier","OrganizationIdentifier","GeoNameIdentifier","LocaleEnglishNameIdentifier","OsBuild","OsSuite","IsProtected","AutoSampleOptIn","SMode","IeVerIdentifier","Firewall","UacLuaenable","Census_OEMNameIdentifier","Census_OEMModelIdentifier","Census_ProcessorCoreCount","Census_ProcessorManufacturerIdentifier","Census_ProcessorModelIdentifier","Census_SystemVolumeTotalCapacity","Census_HasOpticalDiskDrive","Census_TotalPhysicalRAM","Census_InternalPrimaryDisplayResolutionHorizontal","Census_InternalPrimaryDisplayResolutionVertical","Census_OSBuildNumber","Census_OSBuildRevision","Census_OSInstallLanguageIdentifier","Census_OSUILocaleIdentifier","Census_IsPortableOperatingSystem","Census_IsFlightsDisabled","Census_FirmwareManufacturerIdentifier","Census_FirmwareVersionIdentifier","Census_IsSecureBootEnabled","Census_IsVirtualDevice","Census_IsTouchEnabled","Census_IsPenCapable","Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer","Wdft_RegionIdentifier"],"index":[0],"data":[[0,7,0,53447,1,1,1,141,92213,27,167,227,16299,768,1,0,0,117,1,1,585,190026,2,5,2013,458376,1,4096,1366,768,16299,547,9,34,0,0,556,64570,1,0,0,0,0,0,10]]}'

#### Evaluate the sample input vector by sending an HTTP request
We will query the ACI webservice's scoring endpoint by sending an HTTP POST request that contains the input vector.

In [23]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=inputs, headers=headers)
  print("Response: {}".format(response.text))
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [24]:
dev_scoring_uri = dev_webservice.scoring_uri

In [25]:
dev_prediction = query_endpoint_example(scoring_uri=dev_scoring_uri, inputs=sample_json)

Sending batch prediction request with inputs: {"columns":["IsBeta","RtpStateBitfield","IsSxsPassiveMode","AVProductStatesIdentifier","AVProductsInstalled","AVProductsEnabled","HasTpm","CountryIdentifier","CityIdentifier","OrganizationIdentifier","GeoNameIdentifier","LocaleEnglishNameIdentifier","OsBuild","OsSuite","IsProtected","AutoSampleOptIn","SMode","IeVerIdentifier","Firewall","UacLuaenable","Census_OEMNameIdentifier","Census_OEMModelIdentifier","Census_ProcessorCoreCount","Census_ProcessorManufacturerIdentifier","Census_ProcessorModelIdentifier","Census_SystemVolumeTotalCapacity","Census_HasOpticalDiskDrive","Census_TotalPhysicalRAM","Census_InternalPrimaryDisplayResolutionHorizontal","Census_InternalPrimaryDisplayResolutionVertical","Census_OSBuildNumber","Census_OSBuildRevision","Census_OSInstallLanguageIdentifier","Census_OSUILocaleIdentifier","Census_IsPortableOperatingSystem","Census_IsFlightsDisabled","Census_FirmwareManufacturerIdentifier","Census_FirmwareVersionIdentifier","Census_IsSecureBootEnabled","Census_IsVirtualDevice","Census_IsTouchEnabled","Census_IsPenCapable","Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer","Wdft_RegionIdentifier"],"index":[0],"data":[[0,7,0,53447,1,1,1,141,92213,27,167,227,16299,768,1,0,0,117,1,1,585,190026,2,5,2013,458376,1,4096,1366,768,16299,547,9,34,0,0,556,64570,1,0,0,0,0,0,10]]}
Response: [1]
Received response: [1]

## Deploying the model to production using [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/en-us/services/kubernetes-service/)

### Option 1: Create a new AKS cluster

If you do not have an active AKS cluster for model deployment, you can create one using the Azure ML SDK.

In [28]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_cluster_name = "cph-prod" 
# Create the cluster
aks_target = ComputeTarget.create(workspace = workspace, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating..................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None

### Option 2: Connect to an existing AKS cluster

If you already have any active AKS cluster running, you can add it to your Workspace using the Azure ML SDK.

In [30]:
from azureml.core.compute import AksCompute, ComputeTarget

# Get the resource id from https://porta..azure.com -> Find your resource group -> click on the Kubernetes service -> Properties
#resource_id = "/subscriptions/<your subscription id>/resourcegroups/<your resource group>/providers/Microsoft.ContainerService/managedClusters/<your aks service name>"
resource_id = "/subscriptions/%s/resourcegroups/silv-workshop-infra-rg/providers/Microsoft.ContainerService/managedClusters/cph-k8s" % str(dbutils.secrets.get("workshop_secrets", "subscription-id"))

# Give the cluster a local name
#cluster_name = "<CLUSTER_NAME>"
cluster_name = "mldeploycph"

# Attatch the cluster to your workgroup
#aks_target = AksCompute.attach(workspace=workspace, name=cluster_name, resource_id=resource_id)
attach_config = AksCompute.attach_configuration(resource_group="silv-workshop-infra-rg",
                                                cluster_name="cph-k8s")
compute = ComputeTarget.attach(workspace, cluster_name, attach_config)

# Wait for the operation to complete
compute.wait_for_completion(True)
print(compute.provisioning_state)
print(compute.provisioning_errors)

--------------------------------------------------------------------------- 
 ComputeTargetException Traceback (most recent call last)
 <command-3474256664119230> in <module> () 
 16 
 17 # Wait for the operation to complete 
 ---> 18 compute . wait_for_completion ( True ) 
 19 print ( compute . provisioning_state ) 
 20 print ( compute . provisioning_errors ) 

 /databricks/python/lib/python3.6/site-packages/azureml/core/compute/compute.py in wait_for_completion (self, show_output) 
 403 'Current state is {}'.format(self.provisioning_state))
 404 else : 
 --> 405 raise e 
 406 
 407 def _wait_for_completion ( self , show_output ) : 

 /databricks/python/lib/python3.6/site-packages/azureml/core/compute/compute.py in wait_for_completion (self, show_output) 
 396 'state, current provisioning state: {}\n' 
 397 'Provisioning operation error:\n' 
 --> 398 '{}'.format(self.provisioning_state, error_response))
 399 except ComputeTargetException as e : 
 400 if e . message == 'No operation endpoint' : 

 ComputeTargetException : Compute object provisioning polling reached non-successful terminal state, current provisioning state: Failed
Provisioning operation error:
StatusCode: 404
Message: The specified resource was not found

### Deploy to the model's image to the specified AKS cluster

In [32]:
from azureml.core.webservice import Webservice, AksWebservice

# Set configuration and service name
prod_webservice_name = "cph-prod"
prod_webservice_deployment_config = AksWebservice.deploy_configuration()

# Deploy from image
prod_webservice = Webservice.deploy_from_image(workspace = workspace, 
                                               name = prod_webservice_name,
                                               image = model_image,
                                               deployment_config = prod_webservice_deployment_config,
                                               deployment_target = aks_target)

Creating service

In [33]:
# Wait for the deployment to complete
prod_webservice.wait_for_deployment(show_output = True)

Running...............
SucceededAKS service creation operation finished, operation "Succeeded"

## Querying the deployed model in production

### Load a sample input vector from the dataset

In [36]:
import numpy as np
import pandas as pd
from sklearn import datasets

sparkDF = spark.read.format("delta").load("/databricks_workshop/validate_clean").limit(20)

train_x = sparkDF.toPandas()
sample = train_x.iloc[:1, 1:]
sample_json = sample.to_json(orient="split")
query_input = list(sample.as_matrix().flatten())

/local_disk0/tmp/1560154979230-0/PythonShell.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
 # This import has to be done *before* all other imports that could import MPL. Otherwise,

#### Evaluate the sample input vector by sending an HTTP request
We will query the AKS webservice's scoring endpoint by sending an HTTP POST request that includes the input vector. The production AKS deployment may require an authorization token (service key) for queries. We will include this key in the HTTP request header.

In [38]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=inputs, headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [39]:
prod_scoring_uri = prod_webservice.scoring_uri
prod_service_key = prod_webservice.get_keys()[0] if len(prod_webservice.get_keys()) > 0 else None

In [40]:
prod_prediction = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=sample_json)

Sending batch prediction request with inputs: {"columns":["IsBeta","RtpStateBitfield","IsSxsPassiveMode","AVProductStatesIdentifier","AVProductsInstalled","AVProductsEnabled","HasTpm","CountryIdentifier","CityIdentifier","OrganizationIdentifier","GeoNameIdentifier","LocaleEnglishNameIdentifier","OsBuild","OsSuite","IsProtected","AutoSampleOptIn","SMode","IeVerIdentifier","Firewall","UacLuaenable","Census_OEMNameIdentifier","Census_OEMModelIdentifier","Census_ProcessorCoreCount","Census_ProcessorManufacturerIdentifier","Census_ProcessorModelIdentifier","Census_SystemVolumeTotalCapacity","Census_HasOpticalDiskDrive","Census_TotalPhysicalRAM","Census_InternalPrimaryDisplayResolutionHorizontal","Census_InternalPrimaryDisplayResolutionVertical","Census_OSBuildNumber","Census_OSBuildRevision","Census_OSInstallLanguageIdentifier","Census_OSUILocaleIdentifier","Census_IsPortableOperatingSystem","Census_IsFlightsDisabled","Census_FirmwareManufacturerIdentifier","Census_FirmwareVersionIdentifier","Census_IsSecureBootEnabled","Census_IsVirtualDevice","Census_IsTouchEnabled","Census_IsPenCapable","Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer","Wdft_RegionIdentifier"],"index":[0],"data":[[0,7,0,53447,1,1,1,141,92213,27,167,227,16299,768,1,0,0,117,1,1,585,190026,2,5,2013,458376,1,4096,1366,768,16299,547,9,34,0,0,556,64570,1,0,0,0,0,0,10]]}
Received response: [1]

## Updating the production deployment

### Train a new model
Let's import a different MLFlow model.

In [43]:
model_uri_update = "dbfs:/databricks/mlflow/1622123275403635/e71c7cf2fabd487baf88e018ad282db1/artifacts/lrModelSK"

### Build an Azure Container Image for the new model

In [45]:
import mlflow.azureml

model_image_updated, azure_model_updated = mlflow.azureml.build_image(model_uri=model_uri_update,
                                                                      workspace=workspace, 
                                                                      model_name="wine-rating-model",
                                                                      image_name="wine-model-container-image",
                                                                      description="Sklearn ElasticNet image for rating wines", 
                                                                      tags={},
                                                                      synchronous=False)

Registering model wine-rating-model
Creating image

In [46]:
model_image_updated.wait_for_creation(show_output=True)

Running.
NotStarted......................................................................
Succeeded
Image creation operation finished for image wine-model-container-image:1, operation "Succeeded"

### Deploy the new model's image to the AKS cluster

Using the [azureml.core.webservice.AksWebservice.update()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py#update) function, we will replace the deployment's existing model image with the new model image.

In [48]:
prod_webservice.update(image=model_image_updated)

In [49]:
prod_webservice.wait_for_deployment(show_output = True)

Running.................
SucceededAKS service creation operation finished, operation "Succeeded"

### Query the updated model

In [51]:
prod_prediction_updated = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=sample_json)

Sending batch prediction request with inputs: {"columns":["IsBeta","RtpStateBitfield","IsSxsPassiveMode","AVProductStatesIdentifier","AVProductsInstalled","AVProductsEnabled","HasTpm","CountryIdentifier","CityIdentifier","OrganizationIdentifier","GeoNameIdentifier","LocaleEnglishNameIdentifier","OsBuild","OsSuite","IsProtected","AutoSampleOptIn","SMode","IeVerIdentifier","Firewall","UacLuaenable","Census_OEMNameIdentifier","Census_OEMModelIdentifier","Census_ProcessorCoreCount","Census_ProcessorManufacturerIdentifier","Census_ProcessorModelIdentifier","Census_SystemVolumeTotalCapacity","Census_HasOpticalDiskDrive","Census_TotalPhysicalRAM","Census_InternalPrimaryDisplayResolutionHorizontal","Census_InternalPrimaryDisplayResolutionVertical","Census_OSBuildNumber","Census_OSBuildRevision","Census_OSInstallLanguageIdentifier","Census_OSUILocaleIdentifier","Census_IsPortableOperatingSystem","Census_IsFlightsDisabled","Census_FirmwareManufacturerIdentifier","Census_FirmwareVersionIdentifier","Census_IsSecureBootEnabled","Census_IsVirtualDevice","Census_IsTouchEnabled","Census_IsPenCapable","Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer","Wdft_RegionIdentifier"],"index":[0],"data":[[0,7,0,53447,1,1,1,141,92213,27,167,227,16299,768,1,0,0,117,1,1,585,190026,2,5,2013,458376,1,4096,1366,768,16299,547,9,34,0,0,556,64570,1,0,0,0,0,0,10]]}
Received response: [1]

## Cleaning up the deployments

### Terminate the "dev" ACI webservice

Because ACI manages compute resources on your behalf, deleting the "dev" ACI webservice will remove all resources associated with the "dev" model deployment

In [54]:
dev_webservice.delete()

### Terminate the production AKS webservice

This terminates the real-time serving webservice running on the specified AKS cluster. It **does not** terminate the AKS cluster.

In [56]:
prod_webservice.delete()

### Remove the AKS cluster from the Azure ML Workspace

If the cluster was created using the Azure ML SDK (see **Option 1: Create a new AKS cluster**), removing it from the Azure ML Workspace will terminate the cluster, including all of its compute resources and deployments.

If the cluster was created independently (see **Option 2: Connect to an existing AKS cluster**), it will remain active after removal from the Azure ML Workspace.

In [58]:
aks_target.delete()